# 프로젝트를 위한 EDA 과정
- 데이터셋
  - KITTI ( 7,481장 )
  - VKITTI ()
  - AIHub ( 71,030장 )
- 진행 내용
  - Occlusion 비율에 따른 데이터 분포
  - Class balance 확인을 위한 Class 별 데이터 분포
  - Bounding Box 크기에 따른 결과 비교를 위한 분석

## 사전 진행

한 번만 진행하면 됨.
(오래 걸림 주의)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/data_analysis

/content/drive/MyDrive/data_analysis


In [ ]:
!unzip data_object_label_2.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: training/label_2/001705.txt  
 extracting: training/label_2/000367.txt  
 extracting: training/label_2/006048.txt  
 extracting: training/label_2/005468.txt  
 extracting: training/label_2/007308.txt  
 extracting: training/label_2/001417.txt  
 extracting: training/label_2/006638.txt  
 extracting: training/label_2/002132.txt  
 extracting: training/label_2/002407.txt  
 extracting: training/label_2/007186.txt  
 extracting: training/label_2/004363.txt  
 extracting: training/label_2/005764.txt  
 extracting: training/label_2/003298.txt  
 extracting: training/label_2/002664.txt  
 extracting: training/label_2/002322.txt  
 extracting: training/label_2/005098.txt  
 extracting: training/label_2/007135.txt  
 extracting: training/label_2/001415.txt  
 extracting: training/label_2/000344.txt  
 extracting: training/label_2/005904.txt  
 extracting: training/label_2/007166.txt  
 extracting: training/label_2/000502.txt  
 extracting: train

In [ ]:
!unzip 도로주행영상.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: 도로주행영상/Training/bb/도심로/주간일출/맑음/120_전방/20200925_162638/3_annotations_v001_1/3_20200925_162638_000570_v001_1.xml  
  inflating: __MACOSX/도로주행영상/Training/bb/도심로/주간일출/맑음/120_전방/20200925_162638/3_annotations_v001_1/._3_20200925_162638_000570_v001_1.xml  
  inflating: 도로주행영상/Training/bb/도심로/주간일출/맑음/120_전방/20200925_162638/3_annotations_v001_1/3_20200925_162638_000840_v001_1.xml  
  inflating: __MACOSX/도로주행영상/Training/bb/도심로/주간일출/맑음/120_전방/20200925_162638/3_annotations_v001_1/._3_20200925_162638_000840_v001_1.xml  
  inflating: 도로주행영상/Training/bb/도심로/주간일출/맑음/120_전방/20200925_162638/3_annotations_v001_1/3_20200925_162638_001200_v001_1.xml  
  inflating: __MACOSX/도로주행영상/Training/bb/도심로/주간일출/맑음/120_전방/20200925_162638/3_annotations_v001_1/._3_20200925_162638_001200_v001_1.xml  
  infla

## 데이터 불러오기

In [3]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

In [4]:
path = '/content/drive/MyDrive/hackathon/data_analysis/'
kitti_path = path + 'kitti_label/label_2/'
aihub_path = path + 'aihub_label/Training/bb/도심로/'
vkitti_path = path + 'vkitti_label/'

### KITTI 데이터셋 불러오기

In [5]:
kitti_file_list = os.listdir(kitti_path)

len(kitti_file_list)

7481

In [6]:
# 샘플 확인
kitti_sample = pd.read_csv(kitti_path+kitti_file_list[5], sep=' ', header=None)
kitti_sample
print(kitti_path+kitti_file_list[5])

/content/drive/MyDrive/hackathon/data_analysis/kitti_label/label_2/007292.txt


In [7]:
kitti_sample = kitti_sample.loc[kitti_sample[0] != 'DontCare']
kitti_sample

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Car,0.0,0,-1.58,589.08,176.53,615.80,202.94,1.65,1.67,3.64,-0.54,1.91,47.37,-1.59
1,Car,0.0,1,1.93,235.90,190.63,351.06,248.41,1.43,1.63,4.31,-9.11,1.99,21.06,1.53
2,Car,0.0,1,1.77,426.57,184.20,468.57,210.90,1.44,1.62,4.02,-9.44,2.12,42.07,1.55


In [8]:
len(kitti_sample)

3

In [9]:
kitti_sample[2].value_counts()

1    2
0    1
Name: 2, dtype: int64

### VKITTI 데이터셋 불러오기

In [ ]:
bbox_list = []

for (dirpath, dirnames, filenames) in os.walk(vkitti_path):
    for file in filenames:
        combined_path = os.path.join(dirpath, file)
        if '.DS_Store' in combined_path:
            continue
        elif 'bbox.txt' in combined_path:
            bbox_list += [combined_path]
        else:
            continue

In [ ]:
len(bbox_list)

50

In [ ]:
bbox_list[:5]

['/content/drive/MyDrive/hackathon/data_analysis/vkitti_label/Scene18/rain/bbox.txt',
 '/content/drive/MyDrive/hackathon/data_analysis/vkitti_label/Scene18/30-deg-left/bbox.txt',
 '/content/drive/MyDrive/hackathon/data_analysis/vkitti_label/Scene18/overcast/bbox.txt',
 '/content/drive/MyDrive/hackathon/data_analysis/vkitti_label/Scene18/15-deg-right/bbox.txt',
 '/content/drive/MyDrive/hackathon/data_analysis/vkitti_label/Scene18/clone/bbox.txt']

### AIhub 데이터셋 불러오기

In [10]:
# aihub 경로들 모으기
aihub_file_list = []
for (aihub_dirpath, aihub_dirnames, aihub_filenames) in os.walk(aihub_path):
    for file in aihub_filenames:
        aihub_combined_path = os.path.join(aihub_dirpath, file)
        if '.DS_Store' in aihub_combined_path:
            continue
        else:
            aihub_file_list += [aihub_combined_path]

In [11]:
aihub_file_list[:5]

['/content/drive/MyDrive/hackathon/data_analysis/aihub_label/Training/bb/도심로/야간일몰/맑음/60_전방/20200708_200952/2_annotations_v001_1/2_20200708_200952_003390_v001_1.xml',
 '/content/drive/MyDrive/hackathon/data_analysis/aihub_label/Training/bb/도심로/야간일몰/맑음/60_전방/20200708_200952/2_annotations_v001_1/2_20200708_200952_003240_v001_1.xml',
 '/content/drive/MyDrive/hackathon/data_analysis/aihub_label/Training/bb/도심로/야간일몰/맑음/60_전방/20200708_200952/2_annotations_v001_1/2_20200708_200952_003780_v001_1.xml',
 '/content/drive/MyDrive/hackathon/data_analysis/aihub_label/Training/bb/도심로/야간일몰/맑음/60_전방/20200708_200952/2_annotations_v001_1/2_20200708_200952_003510_v001_1.xml',
 '/content/drive/MyDrive/hackathon/data_analysis/aihub_label/Training/bb/도심로/야간일몰/맑음/60_전방/20200708_200952/2_annotations_v001_1/2_20200708_200952_003660_v001_1.xml']

In [12]:
len(aihub_file_list)

71030

## 데이터 전처리

### KITTI 데이터셋 전처리

1. 라벨 정보가 들어있는 전체 txt파일을 하나의 데이터 프레임으로 합치는 작업
2. Class 중 DontCare 삭제

In [15]:
# .txt 파일을 merge.txt라는 파일에 합치고 저장
with open(path + 'merge.txt', 'w') as outfile:
    for filename in sorted(kitti_file_list):
        with open(kitti_path + filename) as file:        
            outfile.write(file.read())

In [16]:
kitti_label = pd.read_csv(path+'merge.txt', sep=' ', header=None)
kitti_label

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Pedestrian,0.0,0,-0.20,712.40,143.00,810.73,307.92,1.89,0.48,1.20,1.84,1.47,8.41,0.01
1,Truck,0.0,0,-1.57,599.41,156.40,629.75,189.25,2.85,2.63,12.34,0.47,1.49,69.44,-1.56
2,Car,0.0,0,1.85,387.63,181.54,423.81,203.12,1.67,1.87,3.69,-16.53,2.39,58.49,1.57
3,Cyclist,0.0,3,-1.65,676.60,163.95,688.98,193.93,1.86,0.60,2.02,4.59,1.32,45.84,-1.55
4,DontCare,-1.0,-1,-10.00,503.89,169.71,590.61,190.13,-1.00,-1.00,-1.00,-1000.00,-1000.00,-1000.00,-10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51860,Car,0.0,2,-2.91,382.70,179.71,495.48,219.10,1.48,1.63,4.10,-6.79,1.78,28.78,-3.14
51861,Car,0.0,0,-1.69,631.71,179.85,678.40,214.37,1.41,1.56,4.08,1.90,1.74,31.98,-1.63
51862,Car,0.0,2,-1.62,619.92,176.90,647.01,199.92,1.43,1.58,3.37,1.47,1.71,47.02,-1.59
51863,Van,0.0,2,-1.58,605.20,168.44,621.13,187.32,2.33,1.96,6.11,0.39,1.83,93.03,-1.57


In [17]:
# DontCare 삭제
kitti_label = kitti_label.loc[kitti_label[0] != 'DontCare']
kitti_label

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Pedestrian,0.0,0,-0.20,712.40,143.00,810.73,307.92,1.89,0.48,1.20,1.84,1.47,8.41,0.01
1,Truck,0.0,0,-1.57,599.41,156.40,629.75,189.25,2.85,2.63,12.34,0.47,1.49,69.44,-1.56
2,Car,0.0,0,1.85,387.63,181.54,423.81,203.12,1.67,1.87,3.69,-16.53,2.39,58.49,1.57
3,Cyclist,0.0,3,-1.65,676.60,163.95,688.98,193.93,1.86,0.60,2.02,4.59,1.32,45.84,-1.55
8,Misc,0.0,0,-1.82,804.79,167.34,995.43,327.94,1.63,1.48,2.37,3.23,1.59,8.55,-1.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51859,Car,0.0,1,-2.90,344.07,180.05,479.10,224.15,1.49,1.59,4.44,-7.12,1.78,26.03,3.12
51860,Car,0.0,2,-2.91,382.70,179.71,495.48,219.10,1.48,1.63,4.10,-6.79,1.78,28.78,-3.14
51861,Car,0.0,0,-1.69,631.71,179.85,678.40,214.37,1.41,1.56,4.08,1.90,1.74,31.98,-1.63
51862,Car,0.0,2,-1.62,619.92,176.90,647.01,199.92,1.43,1.58,3.37,1.47,1.71,47.02,-1.59


### VKITTI 데이터셋 전처리

In [ ]:
# bbox.txt 파일들 모두 합치기
bbox1 = pd.read_csv(bbox_list[0], delimiter=' ')

for i in range(1, len(bbox_list)):
    bbox2 = pd.read_csv(bbox_list[i], delimiter=' ')
    bbox1 = pd.concat([bbox1, bbox2])
    bbox = bbox1[['trackID', 'left', 'right', 'top', 'bottom', 'truncation_ratio', 'occupancy_ratio']]

bbox

,trackID,left,right,top,bottom,truncation_ratio,occupancy_ratio
0,0,565,595,181,202,0.000000,0.739683
1,0,558,588,181,202,0.000000,0.742857
2,0,557,591,183,204,0.000000,0.761905
3,0,550,584,183,204,0.000000,0.774510
4,0,550,586,183,206,0.000000,0.764493
...,...,...,...,...,...,...,...
13895,86,0,194,212,374,0.671171,0.718181
13896,86,0,205,214,374,0.723684,0.768567
13897,86,0,156,214,374,0.832999,0.690665
13898,86,0,165,218,374,0.872686,0.689277


In [ ]:
# trackID -> className으로 바꿔주기 위한 csv 파일 불러오기
label_index = pd.read_csv('/content/drive/MyDrive/hackathon/data_analysis/CAR_VAN_TRUCK.csv')
label_index

,index,className
0,0,Car
1,1,Car
2,10,Car
3,100,Car
4,101,Car
...,...,...
127,94,Car
128,95,Car
129,96,Car
130,97,Car


In [ ]:
label_index.rename(columns={'index' : 'trackID'}, inplace=True)
label_index

,trackID,className
0,0,Car
1,1,Car
2,10,Car
3,100,Car
4,101,Car
...,...,...
127,94,Car
128,95,Car
129,96,Car
130,97,Car


In [ ]:
# trackID 기준으로 클래스 라벨 데이터프레임과 병합
bbox = pd.merge(bbox, label_index, on='trackID')
bbox

,trackID,left,right,top,bottom,truncation_ratio,occupancy_ratio,className
0,0,565,595,181,202,0.000000,0.739683,Car
1,0,558,588,181,202,0.000000,0.742857,Car
2,0,557,591,183,204,0.000000,0.761905,Car
3,0,550,584,183,204,0.000000,0.774510,Car
4,0,550,586,183,206,0.000000,0.764493,Car
...,...,...,...,...,...,...,...,...
532889,93,0,58,223,261,1.000000,0.203267,Car
532890,93,0,52,222,260,0.591558,0.084008,Car
532891,93,0,40,222,258,0.756260,0.019444,Car
532892,93,0,33,222,259,0.846569,0.011466,Car


In [ ]:
bbox['className'].unique()

array(['Car', 'Van', 'Truck'], dtype=object)

### AIhub 데이터셋 전처리

In [18]:
# 클래스명 리스트 선언
names = ['none_of_the_above',
         'Vehicle_Car',
         'Vehicle_Bus',
         'Vehicle_Motorcycle',
         'Vehicle_Unknown',
         'Pedestrian_Pedestrian',
         'Pedestrian_Bicycle',
         'Lane_White_Dash',
         'Lane_White_Solid',
         'Lane_Yellow_Dash',
         'Lane_Yellow_Solid',
         'Lane_Blue_Dash',
         'Lane_Blue_Solid',
         'TrafficLight_Red',
         'TrafficLight_Yellow',
         'TrafficLight_Green',
         'TrafficLight_Arrow',
         'TrafficLight_RedArrow',
         'TrafficLight_YellowArrow',
         'TrafficLight_GreenArrow',
         'TrafficSign_Speed',
         'TrafficSign_Else',
         'RoadMark_StopLine',
         'RoadMark_Crosswalk',
         'RoadMark_Number',
         'RoadMark_Character',
         'RoadMarkArrow_Straight',
         'RoadMarkArrow_Left',
         'RoadMarkArrow_Right',
         'RoadMarkArrow_StraightLeft',
         'RoadMarkArrow_StraightRight',
         'RoadMarkArrow_Uturn',
         'RoadMarkArrow_Else',
         'FreeSpace'
        ]

names_dict = {'none_of_the_above' : 0,
              'Vehicle_Car' : 1,
              'Vehicle_Bus' : 2,
              'Vehicle_Motorcycle' : 3,
              'Vehicle_Unknown' : 4,
              'Pedestrian_Pedestrian' : 5,
              'Pedestrian_Bicycle' : 6,
              'Lane_White_Dash' : 7,
              'Lane_White_Solid' : 8,
              'Lane_Yellow_Dash' : 9,
              'Lane_Yellow_Solid' : 10,
              'Lane_Blue_Dash' : 11,
              'Lane_Blue_Solid' : 12,
              'TrafficLight_Red' : 13,
              'TrafficLight_Yellow' : 14,
              'TrafficLight_Green' : 15,
              'TrafficLight_Arrow' : 16,
              'TrafficLight_RedArrow' : 17,
              'TrafficLight_YellowArrow' : 18,
              'TrafficLight_GreenArrow' : 19,
              'TrafficSign_Speed' : 20,
              'TrafficSign_Else' : 21,
              'RoadMark_StopLine' : 22,
              'RoadMark_Crosswalk' : 23,
              'RoadMark_Number' : 24,
              'RoadMark_Character' : 25,
              'RoadMarkArrow_Straight' : 26,
              'RoadMarkArrow_Left' : 27,
              'RoadMarkArrow_Right' : 28,
              'RoadMarkArrow_StraightLeft' : 29,
              'RoadMarkArrow_StraightRight' : 30,
              'RoadMarkArrow_Uturn' : 31,
              'RoadMarkArrow_Else' : 32,
              'FreeSpace' : 33,
        }

def name2num(name):
    names_dict = {'none_of_the_above' : 0,
              'Vehicle_Car' : 1,
              'Vehicle_Bus' : 2,
              'Vehicle_Motorcycle' : 3,
              'Vehicle_Unknown' : 4,
              'Pedestrian_Pedestrian' : 5,
              'Pedestrian_Bicycle' : 6,
              'Lane_White_Dash' : 7,
              'Lane_White_Solid' : 8,
              'Lane_Yellow_Dash' : 9,
              'Lane_Yellow_Solid' : 10,
              'Lane_Blue_Dash' : 11,
              'Lane_Blue_Solid' : 12,
              'TrafficLight_Red' : 13,
              'TrafficLight_Yellow' : 14,
              'TrafficLight_Green' : 15,
              'TrafficLight_Arrow' : 16,
              'TrafficLight_RedArrow' : 17,
              'TrafficLight_YellowArrow' : 18,
              'TrafficLight_GreenArrow' : 19,
              'TrafficSign_Speed' : 20,
              'TrafficSign_Else' : 21,
              'RoadMark_StopLine' : 22,
              'RoadMark_Crosswalk' : 23,
              'RoadMark_Number' : 24,
              'RoadMark_Character' : 25,
              'RoadMarkArrow_Straight' : 26,
              'RoadMarkArrow_Left' : 27,
              'RoadMarkArrow_Right' : 28,
              'RoadMarkArrow_StraightLeft' : 29,
              'RoadMarkArrow_StraightRight' : 30,
              'RoadMarkArrow_Uturn' : 31,
              'RoadMarkArrow_Else' : 32,
              'FreeSpace' : 33,
        }
    return names_dict[name]

columns = ['<=0.0001', '<=0.001', '<=0.01', '<=0.1', '0.1<']

In [19]:
len(aihub_file_list)

71030

In [ ]:
# 베이스 표 만들기
table = [[0 for _ in range(5)] for _ in range(len(names))]
table

for xml_path in aihub_file_list:
    # XML parsing
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # 이미지 넓이, 높이 구하기
    img_width = int(root.find('size').find('width').text)
    img_height = int(root.find('size').find('height').text)

    # 객체들 정보 추출
    for ob in root.findall('object'):
        bbox = ob.find('bndbox') # bounding box
        name = ob.find('name').text # class
        xmin = int(bbox.find('xmin').text) / img_width  # normalise
        ymin = int(bbox.find('ymin').text) / img_height # normalise
        xmax = int(bbox.find('xmax').text) / img_width  # normalise
        ymax = int(bbox.find('ymax').text) / img_height # normalise

        # 객체의 면적 구하기
        width = xmax - xmin 
        height = ymax - ymin
        area = width * height

        # 객체의 면적에 따른 테이블에서의 column index 할당
        if area > 0 and area <= 0.0001:
            col_idx = 0
        elif area > 0.0001 and area <= 0.001:
            col_idx = 1
        elif area > 0.001 and area <= 0.01:
            col_idx = 2
        elif area > 0.01 and area <= 0.1:
            col_idx = 3
        elif area > 0.1:
            col_idx = 4

        # 객체 이름에 따른 테이블에서의 row index 할당
        row_idx = int(name2num(name))

        # 더하기
        table[row_idx][col_idx] += 1

In [ ]:
# 원본 데이터프레임 생성 및 확인
aihub_df = pd.DataFrame(table, index=names, columns=columns)
aihub_df

In [ ]:
# KITTI 라벨에 맞춘 데이터프레임 생성
df_for_kitti = pd.DataFrame(data=0, index=['Car', 'Tram', 'Pedestrian', 'Cyclist', 'Etc'], columns=columns)
df_for_kitti

In [ ]:
for idx in aihub_df.index:
    if idx == 'Vehicle_Car':
        df_for_kitti.loc['Car'] += aihub_df.loc[idx]
    elif idx == 'Vehicle_Unknown':
        df_for_kitti.loc['Car'] += aihub_df.loc[idx]
    elif idx == 'Vehicle_Bus':
        df_for_kitti.loc['Tram'] += aihub_df.loc[idx]
    elif idx == 'Pedestrian_Pedestrian':
        df_for_kitti.loc['Pedestrian'] += aihub_df.loc[idx]
    elif idx == 'Pedestrian_Bicycle':
        df_for_kitti.loc['Cyclist'] += aihub_df.loc[idx]
    else:
        df_for_kitti.loc['Etc'] += aihub_df.loc[idx]

In [ ]:
df_for_kitti

## 데이터 분포 확인

### 1. Occlusion 비율에 따른 분석

객체가 가려진 정도를 나타내는 Occlusion 정보를 활용하여 가져진 정도에 따라 결과물에 미치는 영향을 확인하기 위해 가림 비율에 따른 객체 수를 확인했다.
<br><br>



---


- 0 ( x ≤ 0.05 )
- 1 ( 0.05 < x ≤ 0.33) 
- 2 (0.33 < x ≤ 0.66)
- 3 (0.66 < x)



---
<br>
(VKITTI와 AIhub 데이터에 대해서는 occlusion에 대한 정보를 확인할 수가 없어 진행하지 않음)

#### KITTI

In [ ]:
# Occlusion 별 객체 수
kitti_label[2].value_counts()

In [ ]:
kitti_occlusion = pd.value_counts(kitti_label[2].values, sort=True)
kitti_occlusion.plot.bar()
kitti_occlusion.plot.bar(figsize=(10,8), fontsize=15)

### Class에 따른 객체 수 분석

특정 Class에 해당되는 객체 수가 너무 적을 경우, Class imbalance가 발생하게 된다. 객체를 제대로 detect하지 못하거나 다른 Class로 판단을 하는 경우가 생길 수도 있기 때문에 Class에 따른 객체의 수를 확인했다.
<br><br>

---



- Car(Vehicle_Car, Vehicle_Unknown)
- Van
- Truck
- Pedestrian(Pedestrian_Pedestrian)
- Cyclist(Pedestrian_Bicycle)
- Tram(Vehicle_Bus)
- Misc
- Person_sitting
- 기타


---



#### KITTI

In [ ]:
# Class 별 객체 수
kitti_label[0].value_counts()

In [ ]:
kitti_class = pd.value_counts(kitti_label[0].values, sort=True)
kitti_class.plot.bar()
kitti_class.plot.bar(figsize=(10,8), fontsize=15)

#### VKITTI

In [ ]:
# class별 객체 수 구하기
bbox['className'].value_counts()

In [ ]:
vkitti_class = pd.value_counts(bbox['className'].values, sort=True)
vkitti_class.plot.bar()
vkitti_class.plot.bar(figsize=(10,8), fontsize=15)

#### AIhub

In [ ]:
# 새로운 데이터프레임 row별 sum 확인
df_for_kitti.sum(axis=1)

In [ ]:
aihub_class = df_for_kitti.sum(axis=1)
aihub_class.plot.bar()
aihub_class.plot.bar(figsize=(10,8), fontsize=15)

### Bounding Box 면적에 따른 분석

객체의 크기에 따른 Detect 결과를 비교하기 위해 객체 사이즈를 이미지 사이즈로 나눈 값(x)을 범주로 나누어 주고 각 범주에 포함되는 객체의 수를 구한다.
<br><br>

---


(x = bbox_size / image_size)
- 0 < x ≤ 0.0001
- 0.0001 < x ≤ 0.001
- 0.001 < x ≤ 0.01
- 0.01 < x ≤ 0.1
- 0.1 < x


---



#### KITTI

In [ ]:
# bbox의 면적을 kitti_label[15]에 추가
kitti_label[15] = (kitti_label[6]-kitti_label[4])*(kitti_label[7]-kitti_label[5])
kitti_label

In [ ]:
# 이미지 전체 면적
image_size = 1242*375

# 전체 면적에 대한 bbox 면적
kitti_label['bbox'] = kitti_label[15] / image_size
kitti_label

In [ ]:
# 비율에 따라 범주 나누기
bins = [0, 0.0001, 0.001, 0.01, 0.1, 1]
labels = ['~0.01%','0.01%~0.1%', '0.1%~1%', '1%~10%', '10%~']
kitti_label['ratio'] = pd.cut(kitti_label['bbox'], bins, labels=labels)
kitti_label

In [ ]:
# 범주 별 객체 수
kitti_label['ratio'].value_counts()

In [ ]:
kitti_bbox = pd.value_counts(kitti_label['ratio'].values, sort=False)
kitti_bbox.plot.bar()
kitti_bbox.plot.bar(figsize=(10,8), fontsize=15)

#### VKITTI

In [ ]:
## BBOX 확인

image_size = 1242*375

bbox['bbox'] = (bbox['right']-bbox['left'])*(bbox['bottom']-bbox['top']) / image_size
bbox

In [ ]:
bbox_list = [
    (bbox['bbox'] <= 0.0001),
    (bbox['bbox'] > 0.0001) & (bbox['bbox'] <= 0.001),
    (bbox['bbox'] > 0.001) & (bbox['bbox'] <= 0.01),
    (bbox['bbox'] > 0.01) & (bbox['bbox'] <= 0.1),
    (bbox['bbox'] > 0.1)]

bb_level = [0, 1, 2, 3, 4]

bbox['bbox_level'] = np.select(bbox_list, bb_level, default='Not Specified')
bbox

In [ ]:
bbox['bbox_level'].value_counts()

In [ ]:
vkitti_bbox = pd.value_counts(bbox['bbox_level'].values, sort=False)
vkitti_bbox.plot.bar()
vkitti_bbox.plot.bar(figsize=(10,8), fontsize=15)

#### AIhub

In [ ]:
# Etc row를 제외한 데이터들 확인
df_for_kitti.iloc[:-1, :]

In [ ]:
df_for_kitti.iloc[:-1, :].sum()

In [ ]:
aihub_bbox = df_for_kitti.sum(axis=0)
aihub_bbox.plot.bar()
aihub_bbox.plot.bar(figsize=(10,8), fontsize=15)